# FINN Preprocessor

This notebook contains the code to preprocess active fire data for the FINN fire emissions model. 

## 1. User specified Configurations

To run the FINN preprocessor, you should specify the following: 

1. `tag_af`: a tag or name for your active fire dataset, e.g., `'my-af-data'`
2. `af_fnames`: a list of file paths to active fire shape files (e.g., downloaded from [FIRMS](https://firms.modaps.eosdis.nasa.gov/)) 

The output file generated by this notebook will be named "out_<i>tag_af</i>_*.csv".


#### Default settings

To use a small example dataset that is bundled with the FINN preprocessor, you can use the following settings:

```python
tag_af = 'testOTS_092018'

af_fnames = [
    '../sample_datasets/fire/testOTS_092018/fire_archive_M6_23960.shp',
    '../sample_datasets/fire/testOTS_092018/fire_archive_V1_23961.shp',
]
```

#### Custom settings with MODIS and VIIRS

If you have custom shapefile MODIS and VIIRS shapefiles you can use a custom tag, and specify the file path(s) to the active fire data shapefiles:

```python
tag_af = 'custom-job'

af_fname = [
    'path/to/fire_archive_M6_...',
    'path/to/fire_archive_V1_...'
]
```

#### Custom settings with MODIS or VIIRS

Alternatively, if you have just MODIS or VIIRS, you can specify just one file path, e.g.:

```python
tag_af = 'custom-job'

af_fname = [
    'path/to/fire_archive_M6_...'
]
```

In [ ]:
# Edit this cell to specify custom files, or use the defaults for a test run
tag_af = 'testOTS_092018'

af_fnames = [
    '../sample_datasets/fire/testOTS_092018/fire_archive_M6_23960.shp',
    '../sample_datasets/fire/testOTS_092018/fire_archive_V1_23961.shp',
]

This next cell ensures that the file paths point to real files.

In [ ]:
# if a path does not point to a file, this will raise an error
import os
for f in af_fnames:
    message = 'Ensure that ' + f + ' exists, and the path is correct.'
    assert os.path.isfile(f), message

#### Defining a year for MODIS land cover and vegetation data

MODIS land cover types and vegetation continuous field will be downloaded for the region of active fire input. 
Specify the year by changing the value of `year_rst`.

In [ ]:
# Specify the year to use for MODIS land cover and vegetation data here
year_rst = 2017

The rest of code below should run without modification.

## 2. Module imports

In [ ]:
# python libraries
import sys
import os
import re
import glob
import datetime
import subprocess
import shlex
from urllib.parse import urlparse
from importlib import reload
import gdal
import matplotlib.pylab as plt


# finn preproc codes
sys.path = sys.path + ['../code_anaconda']
import downloader
import af_import
import rst_import
import polygon_import
import run_step1
import run_step2
import export_shp
import plotter

## 3. Download raster datasets

### Settings for Land Surface Datasets (land cover, vegetation continuous field, region definieons)

In [ ]:
# tag to identify datasets, automatically set to be modlct_YYYY, modvcf_YYYY
tag_lct = 'modlct_%d' % year_rst
tag_vcf = 'modvcf_%d' % year_rst

# tag for the region number polygon
tag_regnum = 'regnum'

# definition of variables in the raster files
rasters = [
        {
            'tag': tag_lct,
            'kind': 'thematic',
            'variable': 'lct'
        },
        {
            'tag': tag_vcf,
            'kind': 'continuous',
            'variables': ['tree', 'herb', 'bare'],
        },
        {
            'tag': tag_regnum,
            'kind': 'polygons',
            'variable_in': 'region_num',
            'variable': 'regnum',
        },
]

Raster files URL and directories to save data

In [ ]:
# all downloads are stored in following dir
download_rootdir = '../downloads'

In [ ]:
# earthdata's URL for landcover and VCF
is_leap = (year_rst % 4 == 0)
url_lct = 'https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/%d.01.01/' % year_rst
url_vcf = 'https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/%d.03.%02d/' % (year_rst, 5 if is_leap else 6)

ddir_lct = download_rootdir +'/'+ ''.join(urlparse(url_lct)[1:3])
ddir_vcf = download_rootdir +'/'+ ''.join(urlparse(url_vcf)[1:3])

print('LCT downloads goes to %s' % ddir_lct)
print('VCF downloads goes to %s' % ddir_vcf)

Download land cover type raster, <b>only for the tiles needed for the active fire file</b>

In [ ]:
reload(downloader)
downloader.download_only_needed(url = url_lct, droot = download_rootdir, pnts=af_fnames[0])

Verify LCT files' checksum.  If a file is correpted, the file is downloaded again.

In [ ]:
downloader.purge_corrupted(ddir = ddir_lct, url=url_lct)

Do similar for vegetation continuous field data

In [ ]:
downloader.download_only_needed(url = url_vcf, droot = download_rootdir, pnts=af_fnames[0])

In [ ]:
downloader.purge_corrupted(ddir_vcf, url=url_vcf)

## 4. Import raster datasets

Downloaded files need preprocessing, which is to extract the only raster band needed, and also make coordinate system to be WGS84.  Intermediate files are created in following directories.

In [ ]:
workdir_lct = '../proc_rst_%s' % tag_lct
workdir_vcf = '../proc_rst_%s' % tag_vcf
workdir_regnum = '../proc_rst_%s' % tag_regnum

print('LCT preprocessing occurs in %s' % workdir_lct)
print('VCF preprocessing occurs in %s' % workdir_vcf)
print('RegNum preprocessing occurs in %s' % workdir_regnum)

### Import land cover type

First grab hdf file names from the download directory

In [ ]:
search_string = "%(ddir_lct)s/MCD12Q1.A%(year_rst)s001.h??v??.006.*.hdf" % dict(
        ddir_lct = ddir_lct, year_rst=year_rst)
fnames_lct = sorted(glob.glob(search_string))
print('found %d hdf files' % len(fnames_lct) )
if len(fnames_lct) == 0:
    raise RuntimeError("check if downloads are successful and search string to be correct: %s" % search_string)

The next command performs three tasks, "merge", "resample" and "import".  First two task creates intermediate GeoTiff files in <i>work_dir</i>.  Last task actually import the data into database's <i>raster</i> schema.

In [ ]:
reload(rst_import)

rst_import.main(tag_lct, fnames=fnames_lct, workdir = workdir_lct)

At this point you should able to see the raster in the database using QGIS.  
I am also trying to make quick check here creating simple image for QA, but use of GIS tool is encouraged.

In [ ]:
%matplotlib inline
import plotter
reload(plotter)
try:
    plotter.plot('raster.o_32_rst_%s' % tag_lct, '../code_anaconda/modlct.clr')
except Exception as e:
    print("Got this error: " + str(e))
    print("Didn't work, use QGIS!")
    pass

### Import vegetation continuous fields

Analogous steps repeated for vegetation continous fields.

In [ ]:
# grab hdf file names
search_string = "%(ddir_vcf)s/MOD44B.A%(year)s065.h??v??.006.*.hdf" % dict(
        ddir_vcf = ddir_vcf, year=year_rst)
fnames_vcf = sorted(glob.glob(search_string))
print('found %d hdf files' % len(fnames_vcf) )
if len(fnames_vcf) == 0:
    raise RuntimeError("check if downloads are successfull and search string to be correct: %s" % search_string)

In [ ]:
reload(rst_import)
rst_import.main(tag_vcf, fnames=fnames_vcf, workdir = workdir_vcf)

In [ ]:
%matplotlib inline
import plotter
reload(plotter)
try:
    plotter.plot('raster.o_32_rst_%s' % tag_vcf)
except Exception as e:
    print("Got this error: " + str(e))
    print("Didn't work, use QGIS!")
    pass

### Import countries of the world shapefile

This is actually not a raster but vector data of polygons.  But since it serves conceptually similar function as raster (specify attribute for a given geographic location), I treat it as if it is a raster dataset.  

In [ ]:
if not os.path.exists(os.path.join(workdir_regnum, 'All_Countries.shp')):
    subprocess.run(['wget', '-P', workdir_regnum, 
                    'https://s3-us-west-2.amazonaws.com/earthlab-finn/All_Countries.zip'], 
                   check=True)
    subprocess.run(['unzip', os.path.join(workdir_regnum, 'All_Countries.zip'), '-d' , workdir_regnum ], 
                  check=True)

In [ ]:
reload(polygon_import)
polygon_import.main('regnum', shpname = os.path.join(workdir_regnum, 'All_Countries.shp'))

## 5. Process active fire data

### Test active fire data files exist

This particular sample AF dataset are provided by FINN developper.  In other applications, it will be user's resoponsibility to provide shape file for active fire in specified path/name.

In [ ]:
print('checking if input files exist:')
re_shp = re.compile('fire_archive_(.*).shp')
re_zip = re.compile('DL_FIRE_(.*).shp')


for i,af_fname in enumerate(af_fnames):
    print("%s: " % af_fname, end='')
    
    pn,fn = os.path.split(af_fname)
    zname = None
    
    if os.path.exists(af_fname):
        print("exists.")
        # if .zip file, need to expand.
        if af_fname[-4:] == '.shp':
            # you are good
            pass
        elif af_fname[-4:] == '.zip':
            # still need to unzip
            zname = af_fname
            m = re_zip.match(af_fname)
            if m:
                arcname = m.group()[0]
                sname = 'fire_archive_%s.shp' % arcname
            else:
                # i cannot predict name of shp file...
                import zipfile
                # find what shp file included...?
                arcname,sname = None, None
        else:
            raise RuntimeError('need to specify .shp file name')
    else:
        print("doesn't exist.")
        
        if af_fname[-4:] == '.shp':
            # guess zip file name
            pn,fn=os.path.split(af_fname)
            m = re_shp.match(fn)
            if m:
                archname = m.groups()[0]
                zname = os.path.join( pn, 'DL_FIRE_%s.zip' % arcname)
                sname = fn
                print('  found zip: %s' % zname)
            else:
                raise RuntimeError('cannot find file: %s' % af_fname)
        else:
            raise RuntimeError('check file exists.')
    if zname:
        subprocess.run(['unzip', '-uo', zname, '-d', os.path.dirname(zname)],
                      check=True)
        assert os.path.exists(os.path.jon(pn, sname))
        af_fnames[i] = os.path.jon(pn, sname)
        

### Import active fire data

Go ahead and import into database.

<b>Be careful!!</b> The code has no safe guard and wipe the schema for the scheama "af_<i>tag_af</i>" and starts over.  

Let me think the design a bit more for now.

In [ ]:
reload(af_import)

# TODO this is destructive need to safe guard!
# tell user schema is there, list table names and # of row of each.  Ask her to delete manually or something to proceed
af_import.main(tag_af, af_fnames)

print()
for i,fn in enumerate(af_fnames):
    print(fn)
    p = subprocess.run(['psql', '-c', 'select count(*) from "af_%s".af_in_%d;' % (tag_af, i+1)], stdout=subprocess.PIPE)
    print(p.stdout.decode())


### Running "step 1" grouping points

In [ ]:
reload(run_step1)
run_step1.main(tag_af, ver='v7m')

### Running "step 2" intersection with raster datasets

In [ ]:
reload(run_step2)

assert run_step2.ver == 'v8b'
run_step2.main(tag_af, rasters)

## 5. Export the output

Default output directory is this diretory (where you have this Jupyter Notebook file), and output file has long name of having tag of each datasets.

In [ ]:
outdir = '.'
shpname = 'out_{0}_{1}_{2}_{3}.shp'.format(tag_af, tag_lct, tag_vcf, tag_regnum)

In [ ]:
schema = 'af_' + tag_af
tblname = 'out_{0}_{1}_{2}'.format(tag_lct, tag_vcf, tag_regnum)
flds = ('v_lct', 'f_lct', 'v_tree', 'v_herb', 'v_bare', 'v_regnum')

**[TODO]** tell where these file went in the file sytem

In [ ]:
reload(export_shp)
export_shp.main(outdir, schema, tblname, flds, shpname)